# Train a model

In [1]:
import tensorflow as tf
import numpy as np

Limited tf.compat.v2.summary API due to missing TensorBoard installation
Limited tf.summary API due to missing TensorBoard installation


## Be sure to used Tensorflow 2.0

In [2]:
assert hasattr(tf, "function") # Be sure to use tensorflow 2.0

## Start by creating a simple model

In [3]:
# Flatten
model = tf.keras.models.Sequential()
# Add layers
model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.Dense(128, activation="relu"))
model.add(tf.keras.layers.Dense(2, activation="softmax"))

## Setup the optimizer,  loss function and one metrics

In [4]:
# The loss method
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
# The optimize
optimizer = tf.keras.optimizers.Adam()
# This metrics is used to track the progress of the training loss during the training
train_loss = tf.keras.metrics.Mean(name='train_loss')

## Create a method which used autograph to train the model

In [5]:
@tf.function
def train_step(image, targets):
    with tf.GradientTape() as tape:
        # Make a prediction
        predictions = model(image)
        # Get the error/loss using the loss_object previously defined
        loss = loss_object(targets, predictions)
    # Compute the gradient which respect to the loss
    gradients = tape.gradient(loss, model.trainable_variables)
    # Change the weights of the model
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    # The metrics are accumulate over time. You don't need to average it yourself.
    train_loss(loss)

## Train the model

In [6]:
for epoch in range(0, 10):
    for _ in range(0, 10):
        # Create fake inputs with two class
        # Create fake inputs
        inputs = np.zeros((2, 30))
        inputs[0] -= 1
        inputs[1] = 1
        # Create fake targets
        targets = np.zeros((2, 1))
        targets[0] = 0
        targets[1] = 1
        # Train the model
        train_step(inputs, targets)
    print("Loss: %s" % train_loss.result())
    train_loss.reset_states()

Loss: tf.Tensor(0.1528348, shape=(), dtype=float32)
Loss: tf.Tensor(0.0039809044, shape=(), dtype=float32)
Loss: tf.Tensor(0.0005385667, shape=(), dtype=float32)
Loss: tf.Tensor(0.00018396397, shape=(), dtype=float32)
Loss: tf.Tensor(0.000104101564, shape=(), dtype=float32)
Loss: tf.Tensor(7.665776e-05, shape=(), dtype=float32)
Loss: tf.Tensor(6.355264e-05, shape=(), dtype=float32)
Loss: tf.Tensor(5.551449e-05, shape=(), dtype=float32)
Loss: tf.Tensor(4.9656337e-05, shape=(), dtype=float32)
Loss: tf.Tensor(4.493916e-05, shape=(), dtype=float32)


# IMPORTANT
Because we used graph mode during the training we can't anymore used the model with model.predict but only in eager mode.

In [7]:
try:
    input_ = np.zeros((1, 30)) + 1
    model.predict(input_)
except Exception as e:
    print("error===>", e)

error===> Unknown graph. Aborting.


However, we can still used the eager mode!

In [10]:
input_ = np.zeros((1, 30)) + 1
model(input_)

<tf.Tensor: id=1312, shape=(1, 2), dtype=float64, numpy=array([[5.99319538e-05, 9.99940068e-01]])>